# InternVL3-8B with 8-bit Quantization for 4x V100 (16GB each)

In [1]:
from pathlib import Path
import random
import math

import numpy as np
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

# Set random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
print("✅ Random seed set to 42 for reproducibility")

✅ Random seed set to 42 for reproducibility


In [2]:
# Update this path to your local InternVL3-8B model
model_id = "/home/jovyan/nfs_share/models/InternVL3-8B"
# Update this path to your test image
imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/image_008.png"

print("🔧 Loading InternVL3-8B with 8-bit quantization for 4x V100 GPUs...")

# 8-bit quantization config for V100 memory efficiency
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)

# Load model with 8-bit quantization and device_map
model = AutoModel.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",  # Required for quantization
    trust_remote_code=True
).eval()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id, 
    trust_remote_code=True, 
    use_fast=False
)

print("✅ Model and tokenizer loaded successfully with 8-bit quantization")
print(f"✅ Model distributed across devices: {model.hf_device_map}")

🔧 Loading InternVL3-8B with 8-bit quantization for 4x V100 GPUs...
FlashAttention2 is not installed.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model and tokenizer loaded successfully with 8-bit quantization
✅ Model distributed across devices: {'vision_model': 0, 'language_model.model.embed_tokens': 0, 'language_model.model.layers.0': 0, 'language_model.model.layers.1': 0, 'language_model.model.layers.2': 0, 'language_model.model.layers.3': 0, 'language_model.model.layers.4': 0, 'language_model.model.layers.5': 0, 'language_model.model.layers.6': 0, 'language_model.model.layers.7': 0, 'language_model.model.layers.8': 0, 'language_model.model.layers.9': 1, 'language_model.model.layers.10': 1, 'language_model.model.layers.11': 1, 'language_model.model.layers.12': 1, 'language_model.model.layers.13': 1, 'language_model.model.layers.14': 1, 'language_model.model.layers.15': 1, 'language_model.model.layers.16': 1, 'language_model.model.layers.17': 1, 'language_model.model.layers.18': 1, 'language_model.model.layers.19': 1, 'language_model.model.layers.20': 1, 'language_model.model.layers.21': 1, 'language_model.model.layers.22': 

In [3]:
# Official InternVL3 image preprocessing (from docs)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

# Process image with proper device handling for 8-bit quantized model
print("🖼️  Processing image...")

# Load image and keep on CPU initially
pixel_values = load_image(imageName, input_size=448, max_num=12)

# For 8-bit quantized models, we need to match the model's dtype
# Check what dtype the vision model expects
vision_device = 'cuda:0'  # Vision model is on GPU 0 based on device_map
model_dtype = torch.float16

print(f"🔍 Model dtype: {model_dtype}")
print(f"🎯 Target device: {vision_device}")

# Convert to model's expected dtype and device
pixel_values = pixel_values.to(device=vision_device, dtype=model_dtype)

print(f"✅ Image processed: {pixel_values.shape}")
print(f"✅ Number of image tiles: {pixel_values.shape[0]}")
print(f"✅ Pixel values dtype: {pixel_values.dtype}")
print(f"✅ Pixel values on device: {pixel_values.device}")
print(f"✅ Vision tokens: {pixel_values.shape[0] * 256}")

# Visual Question Answering - ask a simple question about the image
prompt = 'You are an expert document analyzer specializing in bank statement extraction. Extract the transaction data from this Australian bank statement.'
# InternVL3 format: <image>\n + question
formatted_question = f'<image>\n{prompt}'
print(f"❓ Question: {prompt}")

🖼️  Processing image...
🔍 Model dtype: torch.float16
🎯 Target device: cuda:0
✅ Image processed: torch.Size([7, 3, 448, 448])
✅ Number of image tiles: 7
✅ Pixel values dtype: torch.float16
✅ Pixel values on device: cuda:0
✅ Vision tokens: 1792
❓ Question: You are an expert document analyzer specializing in bank statement extraction. Extract the transaction data from this Australian bank statement.


In [4]:
# Deterministic generation config with multi-turn support
generation_config = dict(
    max_new_tokens=2000,
    do_sample=False  # Pure greedy decoding for deterministic output
)

# Clear CUDA cache before generation
torch.cuda.empty_cache()

# Generate response using InternVL3 API
print("🤖 Generating response with InternVL3-8B (8-bit quantized)...")
print(f"💾 GPU Memory before generation:")
for i in range(torch.cuda.device_count()):
    allocated = torch.cuda.memory_allocated(i) / 1e9
    reserved = torch.cuda.memory_reserved(i) / 1e9
    print(f"   GPU {i}: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")

try:
    # With device_map, model is not wrapped - call chat() directly
    response, history = model.chat(
        tokenizer, 
        pixel_values, 
        formatted_question, 
        generation_config,
        history=None,
        return_history=True
    )
    
    print("✅ Response generated successfully!")
    print("\n" + "="*60)
    print("INTERNVL3 RESPONSE:")
    print("="*60)
    print(response)
    print("="*60)
    
except Exception as e:
    print(f"❌ Error during inference: {e}")
    print(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()
finally:
    # Clean up GPU memory
    torch.cuda.empty_cache()

🤖 Generating response with InternVL3-8B (8-bit quantized)...
💾 GPU Memory before generation:
   GPU 0: 3.51GB allocated, 3.63GB reserved
   GPU 1: 5.55GB allocated, 6.09GB reserved


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


✅ Response generated successfully!

INTERNVL3 RESPONSE:
![](https://i.imgur.com/4ZjKj5r.png)

Here's the extracted transaction data from the Commonwealth Bank statement:

| Date       | Description                                      | Withdrawal | Deposit | Balance     |
|------------|--------------------------------------------------|------------|---------|-------------|
| 07/09/2025 | EFTPOS Cash Out PRICELINE PHARMACY MACKAY QLD     | $322.18    |         | $4880.58    |
| 08/09/2025 | EFTPOS Purchase OFFICEWORKS BUSINESS ROCKHAMPTO...| $64.33     |         | $4921.76    |
| 09/09/2025 | Mortgage Repayment                                | $849.79    |         | $4927.70    |
| 03/09/2025 | OSKO Payment to MIKE CHEN 80884985773133          | $1385.43   |         | $4926.88    |
| 03/09/2025 | Direct Debit 15595481802039 MHF 42730            | $190.17    |         | $5132.31    |
| 02/09/2025 | Salary Payment ATO 287286780782109                |            | $8105.71| $51511.48   |


In [5]:
# Optional: Save response to file
try:
    output_path = Path("internvl3_8b_quantized_output.txt")
    
    with output_path.open("w", encoding="utf-8") as text_file:
        text_file.write(response)
    
    print(f"✅ Response saved to: {output_path}")
    print(f"📄 File size: {output_path.stat().st_size} bytes")
    
except NameError:
    print("❌ Error: 'response' variable not defined.")
    print("💡 Please run the previous cell first to generate the response.")
    
except Exception as e:
    print(f"❌ Error saving file: {e}")

✅ Response saved to: internvl3_8b_quantized_vqa_output.txt
📄 File size: 4168 bytes


## Multi-Turn Conversation Example

InternVL3 supports multi-turn conversations using the history parameter:

In [6]:
# Second turn conversation (uses history from first turn)
try:
    follow_up_question = "Can you provide the total amount from the document?"
    print(f"❓ Follow-up question: {follow_up_question}")
    
    # Use history from previous turn
    response2, history = model.chat(
        tokenizer,
        None,  # No new image for follow-up
        follow_up_question,
        generation_config,
        history=history,  # Pass previous history
        return_history=True
    )
    
    print("\n" + "="*60)
    print("FOLLOW-UP RESPONSE:")
    print("="*60)
    print(response2)
    print("="*60)
    
except NameError:
    print("❌ Error: 'history' variable not defined.")
    print("💡 Please run the first generation cell to create conversation history.")
except Exception as e:
    print(f"❌ Error during follow-up: {e}")

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


❓ Follow-up question: Can you provide the total amount from the document?

FOLLOW-UP RESPONSE:
![](https://i.imgur.com/4ZjKj5r.png)

The total amount from the document is $61,339.58. This is the final balance shown on the bank statement.
